### Importation of Necessary Libraries

In [5]:
import I2C_LCD_driver_library # imports the I2C LCD Driver to interact and control I2C LCD module

from time import * # imports all functions from the "time" library

import sounddevice as sd # imports sounddevice library for handling audio input/output

from scipy.io.wavfile import write, read # imports write and read functions from scipy to save and read audio data respectively

from gpiozero import Button # imports button class from GPIO library for handling GPIO pins connected to buttons

import numpy as np # imports numpy library for handling the audio data as arrays

from datetime import datetime # imports the datetime module to generate timestamps for file names 

import spitch_translator_api_integration as sp # imports custom "spitch_translator" library for speech-to-text-to-speech conversion

### Initialization of Components
This section initializes the LCD object `mylcd` for interacting with the I2C LCD and displays the messages "Welcome!" on the first row and "Press to Record" on the second row of the 16x2 LCD. It also configures GPIO pin 25 for the button and sets up audio recording parameters, including a sampling rate of 44.1 kHz (CD-quality audio), mono recording (single channel), and an empty list `AUDIO_DATA` to store recorded audio chunks.


In [ ]:
mylcd = I2C_LCD_driver_library.lcd() # initializes an object "mylcd" for interacting with the LCD

# LCD INITIAL DISPLAY

mylcd.lcd_display_string("Welcome!", 1) # displays "Welcome" on first row of the 16 x 2 LCD
mylcd.lcd_display_string("Press to Record", 2) # displays "Press to Record" on the second row of the 16 x 2 LCD

# PIN CONFIGURATION

BUTTON_PIN = 25  # assigns GPIO pin 25 to the button

# AUDIO SETTINGS

SAMPLE_RATE = 44100  # sets the sampling rate of the audio record to 44.1 kHz (CD-Quality Audio)
CHANNELS = 1  # sets audio recording to mono recording (single channel)
AUDIO_DATA = []  # initializes an empty list to store recorded audio chunks

### Definition of Necessary Functions


##### 1. `generate_filename`
This section defines a function **generate_filename** that creates unique filenames by combining a base name (defaulting to "hausa_audio") with the current timestamp formatted as **YYYYMMDD_HHMMSS**. The resulting filename is returned with a **.wav** extension. A variable **OUTPUT_FILE** is then initialized to store the generated filename.

In [ ]:
# Function for Generating Filenames

def generate_filename(base_name = "hausa_audio"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") # captures the current date and time in year-month-day_ hour-minute-second format
    return f"{base_name}_{timestamp}.wav" # returms a filename combining base name and timestamp

OUTPUT_FILE = generate_filename() # variable for generated filenames

##### 2. `record_audio_start`
The **record_audio_start** function initiates the process of audio recording. It declares **AUDIO_DATA** as a global variable to ensure access within the function and clears any previously stored data. A message prompts the user that recording has started. Finally, the audio input stream (**stream**) begins capturing audio continuously in chunks.


In [ ]:
# Function to Start Recording

def record_audio_start():
    """Start recording audio."""
    
    global AUDIO_DATA # declares "AUDIO_DATA" as a global variable to access it inside the function
    print("Recording started. Speak into the microphone...")
        
    # STARTS RECORDING CONTINUOUSLY IN CHUNKS
    
    AUDIO_DATA = []  # clears previous data
    stream.start() # starts the audio input stream to capture audio

##### 3. `record_audio_stop`
The function halts the audio recording process and saves the recorded data to a file. It uses **AUDIO_DATA** as a global variable to access the recorded chunks and stops the audio input stream with **stream.stop()**. The audio chunks are combined into a single NumPy array using **np.concatenate** and saved as a **.wav** file with the specified sampling rate. The file is stored at the path specified by **OUTPUT_FILE**, which is also returned by the function.


In [ ]:
# Function to stop recording

def record_audio_stop():
    """Stop recording audio and save to a file."""
    
    global AUDIO_DATA # declares "AUDIO_DATA" as a global variable to access it inside the function
    print("Recording stopped.")
        
    stream.stop() # stops the audio input stream
    final_audio = np.concatenate(AUDIO_DATA, axis=0) # combines all audio chunks into a single Numpy arrays
    write(OUTPUT_FILE, SAMPLE_RATE, final_audio) # saves the recorded audio to a .wav file
    print(f"Recording saved to '{OUTPUT_FILE}'.") 
    
    
    return OUTPUT_FILE # returns the path to the saved audio file

##### 4. `playback_audio`
The **playback_audio** function handles the playback of a **.wav** audio file. It reads the file using the **read** function, extracting the sample rate (**rate**) and audio data (**data**). The audio is then played using the **sd.play** function from the sounddevice library, and the function waits for the playback to finish with **sd.wait()**.


In [ ]:
# Function to Playback Audio

def playback_audio(file_path):
    """Play a .WAV file."""
    
    rate, data = read(file_path) # reads the .wav file into "rate" and "data" variables (i.e for sample rate and audio data)
    sd.play(data, samplerate=rate) # plays the audio using sounddevice library
    sd.wait()  # waits until the saudio playback finishes

##### 5. `button_pressed`
When the button is pressed, the **button_pressed** function is executed. It clears the LCD screen with **mylcd.lcd_clear()**, shows the message **Recording...** on the first row of the 16x2 LCD, and then triggers the **record_audio_start** function to start the audio recording process.


In [ ]:
# Function for Button Press

def button_pressed():
    """Callback function when button is pressed."""
    
    mylcd.lcd_clear() # clears the LCD screen
    mylcd.lcd_display_string("Recording...", 1) # displays "Recording" on first row of the 16 x 2 LCD        
    record_audio_start() # calls function to begin recording

##### 6. `button_released`
The **button_released** function is triggered when the button is released. It starts by clearing the LCD screen and displaying "Record Stop..." on the first row. The function then calls **record_audio_stop** to stop the recording and stores the audio file path. After that, it clears the screen again and shows "Transcribing..." while transcribing the recorded audio from Hausa to English using **sp.speech_to_text**. The transcribed text is then converted into synthesized English audio with **sp.text_to_speech**, and the audio is played back. 

Following this, the transcribed text is displayed on the LCD screen, updating the text in 16-character segments, with a 0.35-second pause between updates. Finally, the screen is cleared and "Thank You!" is displayed, before clearing the screen one last time.


In [ ]:
# Function for Button Release

def button_released():
    """Callback function when button is released."""
    
    mylcd.lcd_clear() # clears the LCD screen
    mylcd.lcd_display_string("Record Stop...", 1) # displays "Record Stop" on first row of the 16 x 2 LCD        
    audio_record = record_audio_stop() # calls "record_audio_stop" function to stop recording and stores the audio file path
    mylcd.lcd_clear() # clears the LCD screen
    mylcd.lcd_display_string("Transcribing...", 1) # displays "Transcribing" on first row of the 16 x 2 LCD
    final_speech = sp.speech_to_text(audio_record, "ha", "en" ) # transcribes the recorded audio from Hausa - ha to English - en
    aud_path = generate_filename("English_audio") # generates a filename for the synthesized audio
    final_audio = sp.text_to_speech(final_speech, 'en', 'lucy', aud_path) # converts the transcribed text into English audio
    print("Second Stage Done")
    playback_audio(aud_path)  # plays the synthesized audio
    
    
    mylcd.lcd_clear() # clears the LCD screen
    text_pad = "" * 16 # creates a string of 16 empty spaces
    text = final_speech # stores the transcribed speech in the "text" variable
    text = text_pad + text 
    
    for i in range (0, len(text)):
        lcd_text = text[i:(i+16)] # extracts slice of text starting at the current index "i" and ending at "i + 16"
        mylcd.lcd_display_string(lcd_text, 1) # displays the extracted text on the first row of the LCD
        sleep(0.35) # pauses the program for 0.35 seconds before updating the LCD screen
        mylcd.lcd_display_string(text_pad, 1) # clears the current row by displaying the empty "text_pad" string.
    
    mylcd.lcd_clear() # clears the LCD screen
    mylcd.lcd_display_string("Thank You!", 1) # displays "Thank You!" on first row of the 16 x 2 LCD
    mylcd.lcd_clear() # clears the LCD screen

This part of the program initializes the GPIO button and assigns it to the specified pin. When the button is pressed, the `button_pressed` function is triggered, and when released, the `button_released` function is called. The button is set to listen for these events and execute the corresponding actions.

Additionally, an audio input stream is configured using the `sounddevice` library. The stream is set to record audio with the specified sample rate and mono channel, and a callback function is used to capture and store the audio data as it arrives.

The program then enters a loop, waiting for button press events while continuously running. It will keep running until the user stops it manually using a keyboard interrupt. Once terminated, the audio stream is stopped, and the program performs cleanup to release resources.


In [ ]:
# INITIALIZE THE GPIO BUTTON
button = Button(BUTTON_PIN) # creates a button object connected to the specific GPIO pin
button.when_pressed = button_pressed # links the "button_pressed" callback to the button press event
button.when_released = button_released # links the "button_released" callback to the button release event

# CREATE A SOUNDDEVICE STREAM
stream = sd.InputStream(
    samplerate=SAMPLE_RATE, 
    channels=CHANNELS,
    dtype='int16',
    callback=lambda indata, frames, time, status: AUDIO_DATA.append(indata.copy())
) # configures an audio input stream with a sample rate, mono channel, and a callback to save audio data

try:
    print("Program ready. Press and hold the button to record audio.")
    
    while True:
        # keep the script running and waiting for button press events
        time.sleep(0.1)
        
except KeyboardInterrupt:
    print("Program terminated by user.")
    
finally:
    # stop the stream and cleanup
    if stream.active:
        stream.stop()
    print("Program terminated.")

In [1]:
print(f"""© 
Rizama Victor Samuel
Omole Ifeoluwa
Ahmad Abubakar Sadiq
Yero Muhammad Bunuyaminu
Ahmad Abdulaziz Ibrahim
""")

© 
Rizama Victor Samuel
Omole Ifeoluwa
Ahmad Abubakar Sadiq
Yero Muhammad Bunuyaminu
Ahmad Abdulaziz Ibrahim

